### 팟빵 다운로드 1단계
* 팟빵 id 와 제목을 함수의 인자로 받는다.
* 첫번째의 8개의 에피소드만 다운로드 받고 있다.
* 저장되는 mp3 파일명을 에피소드의 타이틀로 설정함

In [1]:
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
#문자열에 있는 특수문자를 제거하는 함수
def clean_text(text):
    import re
    text = text.replace("\n", "")
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', text)
    cleaned_text = cleaned_text.replace(' ','')
    return cleaned_text

In [3]:
clean_text("김영철의 파워FM - 진짜 미국식 영어 810회 - 타일러의 진짜 미국식 영어 = Don't be so stubborn. = 억지 좀 부리지 마세요!!")

'김영철의파워FM진짜미국식영어810회타일러의진짜미국식영어Dontbesostubborn억지좀부리지마세요'

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import os

#https://www.podbbang.com/channels/16524
#url = 'http://www.podbbang.com/_m_api/podcasts/16524/episodes?offset=0&sort=pubdate:desc&episode_id=0&limit=8&with=summary&cache=0'
url = 'https://app-api6.podbbang.com/channels/16524/episodes?offset=0&limit=20&sort=desc&episode_id=0&focus_center=0&with=image'
res = requests.get(url)
print(res.status_code)
if res.ok:
    json_data = res.json()    
#     #{data:[{},{},{}]}
    episode_list = json_data['data']
    print(len(episode_list))

    for episode in episode_list:
        print('------')
        #에피소드의 제목
        title = episode['title']
        #에피소드 다운로드 url
        mp3_url = episode['media']['url']
        print(title, mp3_url)
        
        #url 값이 있으면 다운로드 받아야 함
        if mp3_url:            
            req_header = {
                'referer':'https://www.podbbang.com/channels/16524',
                'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
            }
                        
            if not os.path.isdir('mp3'):
                os.mkdir('mp3')
            
            dir_name = os.path.join('mp3','미드처방전')
            #print(dir_name) #mp3\미드처방전

            #해당 디렉토리가 없으면
            if not os.path.isdir(dir_name):
                #디렉토리를 생성
                os.makedirs(dir_name)
            
            #해당 url에서 파일명만 추출
            #file_name = os.path.basename(mp3_url)
            file_name = f'{clean_text(title)}.mp3'
            #생성된 디렉토리명과 파일명을 합쳐주기
            file_name = os.path.join(dir_name,file_name)#dir_name+'/'+file_name            
            print(file_name)

            res = requests.get(mp3_url, headers=req_header)
            print(res.status_code)
            if res.ok:
                #response 객체에서 binary 데이터 추출
                mp3_bin = res.content                
                #binary 데이터를 local 파일로 저장
                with open(file_name, 'wb') as file:
                    file.write(mp3_bin)

200
20
------
120회 억지로 기분 나아지게 하지 맙시다 https://file.ssenhosting.com/data1/pb_22419/watchthegrass.mp3?cmad=2f64617461312f70625f32323431392f776174636874686567726173735f6133312e6d7033
mp3/미드처방전/120회억지로기분나아지게하지맙시다.mp3
200
------
119회 한국 코로나 대응에 대한 외신 뉴스와 반응 https://file.ssenhosting.com/data1/pb_22419/74overseasnews.mp3?cmad=2f64617461312f70625f32323431392f37346f766572736561736e6577735f6133312e6d7033
mp3/미드처방전/119회한국코로나대응에대한외신뉴스와반응.mp3
200
------
118회 이런 콜라보 웃음 폭탄, 영어 아웃 오브 안중 주의 https://file.ssenhosting.com/data1/pb_22419/hand2020226.mp3?cmad=2f64617461312f70625f32323431392f68616e64323032303232365f6133312e6d7033
mp3/미드처방전/118회이런콜라보웃음폭탄영어아웃오브안중주의.mp3
200
------
117회 넷플릭스 본다면 이거슨 꿀팁 '나 홀로 그대' 공동구매각 https://file.ssenhosting.com/data1/pb_22419/handholo1.mp3?cmad=2f64617461312f70625f32323431392f68616e64686f6c6f315f6133312e6d7033
mp3/미드처방전/117회넷플릭스본다면이거슨꿀팁나홀로그대공동구매각.mp3
200
------
116회 넌 라면먹을 계획이 다 있구나, 라끼남 맛 감탄을 영어로 https://file.ssenhosting.com/data1/pb_22419/hand2012.mp3?cmad=2f64617461312f7062

### 팟빵 다운로드 2단계
* 20개의 에피소드를 다운로드 하는 함수로 선언
* 채널 ID : channel_id
* 채널 제목 : channel_title

In [5]:
def podbbang_download(channel_id, channel_title):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urljoin
    import json
    import os

    url = f'https://app-api6.podbbang.com/channels/{channel_id}/episodes'
    #?offset=0&limit=20&sort=desc&episode_id=0&focus_center=0&with=image'
    req_param = {
        "offset":0,
        "limit":20,
        "sort":"desc",
        "episode_id":0,
        "focus_center":0,
        "with":"image"        
    }

    res = requests.get(url, params=req_param)
    print(res.status_code)
    if res.ok:
        json_data = res.json()    
        #{data:[{},{},{}]}
        episode_list = json_data['data']
        print(len(episode_list))

        for episode in episode_list:
            print('------')
            #에피소드의 제목
            title = episode['title']
            #에피소드 다운로드 url
            mp3_url = episode['media']['url']
            print(title, mp3_url)

            #url 값이 있으면 다운로드 받아야 함
            if mp3_url:            
                req_header = {
                    'referer':f'https://www.podbbang.com/channels/{channel_id}',
                    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
                }

                if not os.path.isdir('mp3'):
                    os.mkdir('mp3')

                dir_name = os.path.join('mp3',channel_title)
                #print(dir_name) #mp3\미드처방전

                #해당 디렉토리가 없으면
                if not os.path.isdir(dir_name):
                    #디렉토리를 생성
                    os.makedirs(dir_name)

                #해당 url에서 파일명만 추출
                #file_name = os.path.basename(mp3_url)
                clean_title = clean_text(title)
                file_name = f'{clean_title}.mp3'
                #생성된 디렉토리명과 파일명을 합쳐주기
                file_name = os.path.join(dir_name, file_name)           
                #print(file_name)

                res = requests.get(mp3_url, headers=req_header)
                if res.ok:
                    #response 객체에서 binary 데이터 추출
                    mp3_bin = res.content                
                    #binary 데이터를 local 파일로 저장
                    with open(file_name, 'wb') as file:
                        file.write(mp3_bin)


In [6]:
podbbang_download(15357,'반란의영문법')

200
20
------
[유튜브 강의 오디오 버전] 명사의 종류 https://file.ssenhosting.com/data1/pb_20454/KL02nountypes.mp3?cmad=2f64617461312f70625f32303435342f4b4c30326e6f756e74797065735f6133312e6d7033
------
[유튜브 강의 오디오 버전] of는 물리적 변화, from은 화학적 변화?? https://file.ssenhosting.com/data1/pb_20454/KL01fromof.mp3?cmad=2f64617461312f70625f32303435342f4b4c303166726f6d6f665f6133312e6d7033
------
[유튜브 강의 오디오 버전] 10. 가정법 개념 정리 - 한국과 영어권의 차이 https://file.ssenhosting.com/data1/pb_20454/subjunctive10.mp3?cmad=2f64617461312f70625f32303435342f7375626a756e637469766531305f6133312e6d7033
------
[유튜브 강의 오디오 버전] 9. If ... will ... (if 조건절에 will이 쓰이는 경우들) https://file.ssenhosting.com/data1/pb_20454/condi009.mp3?cmad=2f64617461312f70625f32303435342f636f6e64693030395f6133312e6d7033
------
[유튜브 강의 오디오 버전] 8. as if와 I wish - 한국어와 표현 방식이 같은데 오히려 어려운 이유 https://file.ssenhosting.com/data1/pb_20454/backshift08.mp3?cmad=2f64617461312f70625f32303435342f6261636b736869667430385f6133312e6d7033
------
[유튜브 강의 오디오 버전] 7. unless와 if...not... (

### 팟빵 다운로드 3단계
* 팟빵 id 와 제목을 함수의 인자로 받는다.
* 저장되는 mp3 파일명을 에피소드의 타이틀로 설정함
* 모든 에피소드를 다운로드 받는다.
 - 에피소드를 요청할때 offset 값의 최대값이 항상 변하므로 
 - itertools의 count() 함수를 사용해 무한루프를 수행하도록 한다.
 - 무한루프를 빠져 나올수 있는 조건식을 반드시 주어야 한다.
   - json_data['data'] 길이가 0 이면 무한루프를 탈출한다.
 - 사람이 직접 다운로드 받는 것 처럼 보이게 하기 위해서 sleep time 설정한다.
   - time 이라는 내부모듈의 sleep 함수를 사용한다.

In [7]:
def podbbang_all_download(channel_id, channel_title):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urljoin
    import json
    import os
    from itertools import count
    from time import sleep
    
    url = f'https://app-api6.podbbang.com/channels/{channel_id}/episodes'
    for offset in count(0):
        req_param = {
            "offset":offset,
            "limit":20,
            "sort":"desc",
            "episode_id":0,
            "focus_center":0,
            "with":"image"        
        }

        res = requests.get(url, params=req_param)
        print(offset, res.status_code)
        if res.ok:
            json_data = res.json()    
            #{data:[{},{},{}]}
            episode_list = json_data['data']
            print(len(episode_list))

            # 무한루프 탈출 조건
            if len(json_data['data']) == 0:
               break                 
                
            for episode in episode_list:
                print('------')
                #에피소드의 제목
                title = episode['title']
                #에피소드 다운로드 url
                mp3_url = episode['media']['url']
                print(title, mp3_url)

                #url 값이 있으면 다운로드 받아야 함
                if mp3_url:            
                    req_header = {
                        'referer':f'https://www.podbbang.com/channels/{channel_id}',
                        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
                    }

                    if not os.path.isdir('mp3'):
                        os.mkdir('mp3')

                    dir_name = os.path.join('mp3',channel_title)
                    #print(dir_name) #mp3\미드처방전

                    #해당 디렉토리가 없으면
                    if not os.path.isdir(dir_name):
                        #디렉토리를 생성
                        os.makedirs(dir_name)

                    clean_title = clean_text(title)
                    file_name = f'{clean_title}.mp3'
                    #생성된 디렉토리명과 파일명을 합쳐주기
                    file_name = os.path.join(dir_name, file_name)           
                    #print(file_name)

                    res = requests.get(mp3_url, headers=req_header)
                    if res.ok:
                        #response 객체에서 binary 데이터 추출
                        mp3_bin = res.content                
                        #binary 데이터를 local 파일로 저장
                        with open(file_name, 'wb') as file:
                            file.write(mp3_bin)
                            
                        #0.5초간 프로세스를 중지함, 기계가 아니라 사람처럼 보이게 하려고
                        sleep(0.5)


In [ ]:
podbbang_all_download('16255','여바라')

0 200
20
------
46. 소소하게 여행중독 #2 : 문상건 작가의 아시아와 세이셸 https://file.ssenhosting.com/data1/pb_21610/yeovara46.mp3?cmad=2f64617461312f70625f32313631302f79656f7661726134365f6133312e6d7033
